In [8]:
import sqlite3
#import pathlib untuk mendapatkan direktori file
import pathlib
#import datetime untuk modul mendapatkan tanggal saat ini
import datetime

#Class untuk meng-handle query sql
class DataManager:
    def __init__(self):
        #formula mendapatkan directori database yg disimpan ke variable berupa string
        database = str(pathlib.Path().absolute())+"\Taliabeauty.db"
        #melakukan koneksi pada database
        self.connector = sqlite3.connect(database)
        #inisiasi fungsi cursor database, sehingga dapat menjalankan fungsi query lainnya
        self.cursor = self.connector.cursor()
        
    #fungsi untuk menjalankan query dan menerima return value atau tidak
    def executeQuery(self, query, retVal=False):
        #menjalankan perintah query
        self.cursor.execute(query)
        #menyimpan hasil query ke dalam variable
        all_results = self.cursor.fetchall()
        #mengkonfir1masi perintah query
        self.connector.commit()
        #retVal dipakai kalau def executeQuery perlu mengembalikan nilai atau tidak
        if retVal:
            return all_results

#Class untuk meng-handle login, meng-inherit dari class DataManager
class User(DataManager):
    def __init__(self):
        #menginisiasi atribut DataManager, supaya dapat digunakan dalam class User
        DataManager.__init__(self)
        #beberapa data yang disimpan dalam code yang nantinya juga mengambil data dari query database
        self.id_user = None
        self.email = None
        self.password = None
        self.username = None
        self.tanggal_lahir = None
        self.total = 0
        self.historis_list = []
        
    def login(self, email, password):
        #SELECT untuk mendapatkan variabel2 tertera
        query = 'SELECT id_user, email, password, username, tanggal_lahir FROM user_data \
            where email=\'%s\' and password=\'%s\' '
        #memasukkan Value (%s itu diatas) ke dalam query dari program
        query = query % (email, password)
        #menjalankan function executeQuery yg memiliki nilai retVal True sehingga mengembalikan suatu nilai
        #yang lalu nilai tersebut ditampung di dalam variabel data_login dalam bentuk list/array
        data_login = self.executeQuery(query, True)
        konfirm_login = False
        #mengambil satu2 nilai dari data_login (karena berbentuk array)
        for i in range(0,len(data_login)):
            #mengecek jika data input email dan password terdapat di dalam data_login
            if email == data_login[i][1] and password == data_login[i][2]:
                konfirm_login = True
                self.id_user = data_login[i][0]
                self.email = data_login[i][1]
                self.password = data_login[i][2]
                self.username = data_login[i][3]
                self.tanggal_lahir = data_login[i][4]
                print("Login berhasil")
                print(f"Selamat datang {self.username} di TALIABEAUTY!")
        if konfirm_login == False:
            print(f"Data Login email {email}, password {password} tidak ditemukan, harap untuk melakukan register")

    def register(self, email, password, username, tanggal_lahir):
        #INSERT INTO untuk memasukkan data ke dalam tabel query
        query = 'INSERT INTO user_data (email, password, username, tanggal_lahir) \
                VALUES (\'%s\', \'%s\', \'%s\', \'%s\')'
        #mendapatkan Value untuk dimasukkan data ke dalam tabel query
        query = query % (email, password, username, tanggal_lahir)
        self.executeQuery(query)
        #SELECT untuk mengambil data2 variabel tertera dari tabel query
        query = 'SELECT id_user, email, password FROM user_data \
            where email=\'%s\' and password=\'%s\' '
        query = query % (email, password)
        data_login = self.executeQuery(query, True)
        #mengambil id_user dari data_login dengan email dan password yg sesuai
        for i in range(0,len(data_login)):
            if email == data_login[i][1] and password == data_login[i][2]:
                self.id_user = data_login[i][0]
        #mengubah data sesuai inputan register
        self.email = email
        self.password = password
        self.username = username
        self.tanggal_lahir = tanggal_lahir
        print(f"Akun berhasil dibuat. Selamat datang {self.username} di TALIABEAUTY!")
        
    def daftar_perawatan(self):
        print("Daftar perawatan:")
        print("1. Green Tea Facial (Rp. 150.000)")
        print("2. Silver Facial (Rp. 200.000)")
        
    def daftar_produk(self):
        print("1. Green Tea Extract Bundle (Rp. 250.000)")
        print('2. Green Tea Extract Facial Wash (Rp. 50.000)')
        print('3. Green Tea Extract Facial Cleanser (Rp. 50.000)')
        print('4. Green Tea Extract Toner (Rp. 50.000)')
        print('5. Green Tea Extract Night Cream (Rp. 50.000)')
        print('6. Green Tea Extract Day Cream (Rp. 50.000)')
        
    def reservasi(self, jenis_reservasi, nomor, jumlah=1):
        #Bentuk data produk dalam variabel array, nomor pertama adalah id produk, kedua nama, dan ketiga harga
        perawatan_1 = ['rawat1', 'Green Tea Facial', 150_000]
        perawatan_2 = ['rawat2', 'Silver Facial', 200_000]
        produk_1 = ['produk1', 'Green Tea Extract', 250_000]
        produk_2 = ['produk2', 'Green Tea Extract Facial Wash', 50_000]
        produk_3 = ['produk3', 'Green Tea Extract Facial Cleanser', 50_000]
        produk_4 = ['produk4', 'Green Tea Extract Toner', 50_000]
        produk_5 = ['produk5', 'Green Tea Extract Night Cream', 50_000]
        produk_6 = ['produk6', 'Green Tea Extract Day Cream', 50_000]
        
        #menyiapkan tanggal dan perintah query
        tanggal_pemesanan = datetime.datetime.today().strftime('%Y-%m-%d')
        query = 'INSERT INTO tb_transaksi (id_user, id_barang, nama_barang, jumlah_barang, harga_satuan_barang, tanggal_pemesanan) \
                VALUES (\'%s\', \'%s\', \'%s\', \'%s\', \'%s\', \'%s\')'
        
        #mengecek jenis reservasi, 1 == perawatan, 2 == beli produk, 3 == konsultasi
        #mengecek nomor sesuai dengan yang ada di daftar_produk() dan daftar_perawatan()
        if jenis_reservasi == '1' and nomor == '1':
            query = query % (self.id_user, perawatan_1[0], perawatan_1[1], jumlah, perawatan_1[2], tanggal_pemesanan)
            print(f"{self.username} telah memesan perawatan Green Tea Facial")
            
        elif jenis_reservasi == '1' and nomor == '2':
            query = query % (self.id_user, perawatan_2[0], perawatan_2[1], jumlah, perawatan_2[2], tanggal_pemesanan)
            print(f"{self.username} telah memesan perawatan Silver Facial")
            
        elif jenis_reservasi == '2' and nomor == '1':
            query = query % (self.id_user, produk_1[0], produk_1[1], jumlah, produk_1[2], tanggal_pemesanan)
            print(f"{self.username} telah memesan produk Green Tea Extract sebanyak {jumlah}")
            
        elif jenis_reservasi == '2' and nomor == '2':
            query = query % (self.id_user, produk_2[0], produk_2[1], jumlah, produk_2[2], tanggal_pemesanan)
            print(f"{self.username} telah memesan produk Green Tea Extract Facial Wash sebanyak {jumlah}")
            
        elif jenis_reservasi == '2' and nomor == '3':
            query = query % (self.id_user, produk_3[0], produk_3[1], jumlah, produk_3[2], tanggal_pemesanan)
            print(f"{self.username} telah memesan produk Green Tea Extract Facial Cleaner sebanyak {jumlah}")
            
        elif jenis_reservasi == '2' and nomor == '4':
            query = query % (self.id_user, produk_4[0], produk_4[1], jumlah, produk_4[2], tanggal_pemesanan)
            print(f"{self.username} telah memesan produk Green Tea Extract Toner sebanyak {jumlah}")
            
        elif jenis_reservasi == '2' and nomor == '5':
            query = query % (self.id_user, produk_5[0], produk_5[1], jumlah, produk_5[2], tanggal_pemesanan)
            print(f"{self.username} telah memesan produk Green Tea Extract Night Cream sebanyak {jumlah}")
            
        elif jenis_reservasi == '2' and nomor == '6':
            query = query % (self.id_user, produk_6[0], produk_6[1], jumlah, produk_6[2], tanggal_pemesanan)
            print(f"{self.username} telah memesan produk Green Tea Extract Day Cream sebanyak {jumlah}")
            
        self.executeQuery(query)
        
    def historis(self):
        query = 'SELECT trans.tanggal_pemesanan, user.username, trans.nama_barang, trans.jumlah_barang, trans.harga_satuan_barang \
            FROM tb_transaksi trans \
            JOIN user_data user on trans.id_user = user.id_user'
        #data hasil query disimpan di dalam variabel data_transaksi
        data_transaksi = self.executeQuery(query, True)
        print(f"Riwayat pemesanan {self.username}:")
        total_seluruh = 0
        #mengambil data dari variabel data_transaksi
        for i in data_transaksi:
            #pada indeks ke-3 dan ke-4 dari variabel array i masing-masing adalah jumlah_barang dan harga satuan
            #indeks ke-3 dan ke-4 dikalikan untuk mendapatkan total harga, lalu += untuk menambahkan
            #harga total di tiap perulangan menjadi total keseluruhan
            total_seluruh += i[3]*i[4]
            #indeks ke-0 adalah tanggal_pemesanan, indeks ke-1 adalah username ... (sesuai query Line 157)
            print(f"{i[0]} {i[2]}, Jumlah: {i[3]}, Harga: Rp{i[4]:,}, Total: Rp{(i[3]*i[4]):,}")
        print(f"Total keseluruhan: Rp{total_seluruh:,}")
                
    def akun(self):
        print(f"email: {self.email}\npassword: {self.password}\nusername: {self.username}\ntanggal lahir: {self.tanggal_lahir}")

#menginisiasi (Instantiate) pembuatan objek user
u = User()
print("Selamat datang di TALIABEAUTY")
print("(Masukkan menu dengan menginputkan angka)")

#memulai menerima inputan dari user
while True:
    print("1. Login\n2. Register\n3. Reservasi\n4. Historis\n5. Akun\n6. Exit")
    #lower() dipakai supaya inputan menjadi huruf kecil semua (lowercase)
    menu = input("Menu >").lower()
    
    #menu Login
    if menu == '1':
        print("------Login------")
        email = input("Masukkan email:")
        password = input("Masukkan password:")
        u.login(email,password)
        
    #menu Register
    elif menu == '2':
        print("------Register------")
        email = input("Masukkan email:")
        password = input("Masukkan password:")
        username = input("Masukkan username:")
        tanggal_lahir = input("Masukkan tamggal lahir (ex. 26 Juli 2001):")
        u.register(email, password, username, tanggal_lahir)
        
    #menu Reservasi
    elif menu == '3':
        print("------Reservasi------")
        #if u.username != None mengecek jika sudah ada login yg dilakukan (karena awal program, username bernilai
        #bernilai 'None' yang berarti belum ada data login
        if u.username != None:
            print("1. Perawatan\n2. Beli produk\n3. Konsultasi")
            jenis_reservasi = input("Menu Reservasi:")
            
            if jenis_reservasi == '1':
                u.daftar_perawatan()
                nomor = input("Pilih nomor perawatan:")
                u.reservasi(jenis_reservasi, nomor)
                
            elif jenis_reservasi == '2':
                u.daftar_produk()
                nomor = input("Pilih nomor produk:")
                jumlah = int(input("Jumlah produk dibeli:"))
                u.reservasi(jenis_reservasi, nomor, jumlah)
                
            elif jenis_reservasi == '3':
                print("Konsultasi masalah kecantikan:")
                print("1. Bagaimana mengatasi mata panda?")
                print("2. Bagaimana mengatasi pori-pori di wajah?")
                print("3. Bagaimana menghilangkan bekas jerawat?")
                tanya = input("nomor pertanyaan >")
                if tanya == '1':
                    print("Jawab: Ada beberapa cara menghilangkannya, seperti firming, brightening eye creams\ndan teknik facial massage.")
                elif tanya == '2':
                    print("Jawab: Menurut para dermatologis, diameter pori-pori seseorang tida bisa benar-benar menyusut.\nMeskipun demikian, penampakannya bisa diminimalisasi dengan menggunakan produk tertentu.\nSelain itu, memakai skincare secara rutin juga bisa menjadi salah satu cara ampuhnya.")
                elif tanya == '3':
                    print("Jawab: Salah satu perawatan yang sedang populer, yakni chemical peels, yang mengexfoliate lapisan\natas kulit agar terlihat lebih halus.")
                else:
                    print("Pilih dengan memasukkan angka menu")
                    
            else:
                print("Pilih dengan memasukkan angka menu")
        else:
            print("Diperlukan Login terlebih dahulu")
            
    #menu Historis
    elif menu == '4':
        print("------Historis------")
        if u.username != None:
            u.historis()
        else:
            print("Diperlukan Login terlebih dahulu")
        
    #menu Akun
    elif menu == '5':
        print("------Akun------")
        u.akun()
        
    #menu Exit
    elif menu == '6':
        print("------Selamat Tinggal------")
        print("Terima kasih telah mengunjungi TALIABEAUTY")
        break
        
    #daftar menu
    elif menu == 'menu':
        print("(Masukkan menu dengan menginputkan angka)")
        print("1. Login\n2. Register\n3. Historis\n4. Reservasi\n5. Akun\n6. Exit")
        
    #ketika input menu tidak diketahui
    else:
        print("ketik menu untuk melihat daftar menu")

Selamat datang di TALIABEAUTY
(Masukkan menu dengan menginputkan angka)
1. Login
2. Register
3. Reservasi
4. Historis
5. Akun
6. Exit


Menu > 1


------Login------


Masukkan email: taliabeauty@gmail.com
Masukkan password: talia


Login berhasil
Selamat datang Owner Talia di TALIABEAUTY!
1. Login
2. Register
3. Reservasi
4. Historis
5. Akun
6. Exit


...

Konsultasi masalah kecantikan:
1. Bagaimana mengatasi mata panda?
2. Bagaimana mengatasi pori-pori di wajah?
3. Bagaimana menghilangkan bekas jerawat?


nomor pertanyaan > 2


Menurut para dermatologis, diameter pori-pori seseorang tida bisa benar-benar menyusut.
Meskipun demikian, penampakannya bisa diminimalisasi dengan menggunakan produk tertentu.
Selain itu, memakai skincare secara rutin juga bisa menjadi salah satu cara ampuhnya.
1. Login
2. Register
3. Reservasi
4. Historis
5. Akun
6. Exit


Menu > 6


------Selamat Tinggal------
Terima kasih telah mengunjungi TALIABEAUTY
